In [1]:
import os
import mlflow
import pandas as pd
import psycopg
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss
from catboost import CatBoostRegressor

In [2]:
os.environ["DB_DESTINATION_HOST"] = os.getenv("DB_DESTINATION_HOST")
os.environ["DB_DESTINATION_PORT"] = os.getenv("DB_DESTINATION_PORT")
os.environ["DB_DESTINATION_NAME"] = os.getenv("DB_DESTINATION_NAME")
os.environ["DB_DESTINATION_USER"] = os.getenv("DB_DESTINATION_USER")
os.environ["DB_DESTINATION_PASSWORD"] = os.getenv("DB_DESTINATION_PASSWORD")

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "churn_Andrey"
RUN_NAME = "churn_run"
REGISTRY_MODEL_NAME = "churn_model_Andrey"


In [3]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"], 
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "clean_real_estate"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [4]:
df

,flat_id,floor,is_apartment,kitchen_area,living_area,rooms,total_area,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
0,10769,9,false,8.946669,33.000000,2,43.900002,4431,1962,4,55.705067,37.763611,2.64,72,9,true,10000000.0
1,10770,3,false,7.600000,37.299999,2,58.900002,1858,1957,1,55.738945,37.521091,3.00,91,8,true,17950000.0
2,10771,18,false,13.200000,24.000000,1,54.700001,20908,2008,2,55.809341,37.455643,2.80,528,26,true,14200000.0
3,10773,3,false,11.000000,14.700000,1,34.099998,17563,1999,4,55.858074,37.568119,2.75,77,12,true,9950000.0
4,10774,3,false,7.500000,22.500000,2,37.799999,6528,1966,4,55.713547,37.642311,2.64,72,9,true,9650000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118803,141357,16,false,11.000000,18.000000,1,42.000000,22455,2013,4,55.626579,37.313503,2.64,672,25,true,10500000.0
118804,141358,5,false,5.280000,28.330000,2,41.110001,3162,1960,1,55.727470,37.768677,2.48,80,5,false,7400000.0
118805,141359,7,false,5.300000,20.000000,1,31.500000,6513,1966,4,55.704315,37.506584,2.64,72,9,true,9700000.0
118806,141360,15,false,13.800000,33.700001,2,65.300003,23952,2017,4,55.699863,37.939564,2.70,480,25,true,11750000.0


In [5]:
#import joblib
#with open('../mle-project-sprint-1-v001/part2_dvc/models/fitted_model.pkl', 'rb') as f:
#        model=joblib.load(f)

In [6]:
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(df.columns.values.tolist()))
   

In [7]:
df = df.drop(columns=["is_apartment", "has_elevator"])
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["price"]), df["price"], test_size=0.2, random_state=42)

In [8]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

Learning rate set to 0.08408
0:	learn: 73502908.4610949	total: 51.7ms	remaining: 51.7s
1:	learn: 72625375.6947268	total: 56.6ms	remaining: 28.2s
2:	learn: 71880656.5975624	total: 60.2ms	remaining: 20s
3:	learn: 71270718.1692247	total: 64.1ms	remaining: 16s
4:	learn: 70725603.2274661	total: 68ms	remaining: 13.5s
5:	learn: 70258872.7569320	total: 73.8ms	remaining: 12.2s
6:	learn: 69820768.8590350	total: 77.5ms	remaining: 11s
7:	learn: 69440263.1652143	total: 80.3ms	remaining: 9.96s
8:	learn: 69029849.3949814	total: 84ms	remaining: 9.25s
9:	learn: 68764469.2555896	total: 87.3ms	remaining: 8.65s
10:	learn: 68524184.2282427	total: 91.1ms	remaining: 8.19s
11:	learn: 68243124.4983564	total: 94.8ms	remaining: 7.81s
12:	learn: 68036491.2488713	total: 98ms	remaining: 7.44s
13:	learn: 67854413.5886724	total: 101ms	remaining: 7.13s
14:	learn: 67699262.9527939	total: 104ms	remaining: 6.85s
15:	learn: 67303787.9173503	total: 108ms	remaining: 6.63s
16:	learn: 67175629.7169540	total: 111ms	remaining: 

In [9]:
"""
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = confusion_matrix(y_test, prediction,normalize='all').ravel()
auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss
"""

'\nfrom sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss\n# импортируйте необходимые вам модули\n\n# заведите словарь со всеми метриками\nmetrics = {}\n\n# посчитайте метрики из модуля sklearn.metrics\n# err_1 — ошибка первого рода\n# err_2 — ошибка второго рода\n_, err1, _, err2 = confusion_matrix(y_test, prediction,normalize=\'all\').ravel()\nauc = roc_auc_score(y_test, probas)\nprecision = precision_score(y_test, prediction)\nrecall = recall_score(y_test, prediction)\nf1 = f1_score(y_test, prediction)\nlogloss = log_loss(y_test, prediction)\n\n# запишите значения метрик в словарь\nmetrics["err1"] = err1\nmetrics["err2"] = err2\nmetrics["auc"] = auc\nmetrics["precision"] = precision\nmetrics["recall"] = recall\nmetrics["f1"] = f1\nmetrics["logloss"] = logloss\n'

In [10]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
metrics={}

metrics["mae"] = mean_absolute_error(y_test, prediction)
metrics["mape"] = mean_absolute_percentage_error(y_test, prediction)

In [11]:

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata =  {'model_type': 'no_binary'}

/home/andrey/work/MLE/mle-mlflow/.venv_mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None


In [13]:


# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    model_info = mlflow.catboost.log_model( 
			cb_model=model,
            pip_requirements=pip_requirements,
            signature=signature,
            input_example=input_example,
            metadata=metadata,
            #code_path=code_paths,
            await_registration_for=60,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME)


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert (run.info.status =='FINISHED')# ваш код здесь

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt') # ваш код здесь
#os.remove('users_churn.csv') # ваш код здесьcsv

Successfully registered model 'churn_model_Andrey'.
2024/03/28 23:33:58 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_Andrey, version 1
Created version '1' of model 'churn_model_Andrey'.


In [14]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_test)

#assert model_predictions.dtype == int

print(model_predictions[:10])

[18782604.43140471 12311981.69668086 13237828.82873261 13702443.17831818
 13917359.28308922  5914666.77322041  7914986.47937611  5308258.12713004
 11101393.95720381  8814296.04773738]
